# **말뭉치 프로젝트**
<br>

## **1. train.py 변환**
<br>

#### **1) 라이브러리 및 패키지 가져오기**
<br>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 16.5 MB/s eta 0:00:00


In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00


In [3]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 3.7 MB/s eta 0:00:00


<br>

In [4]:
import sys
import os
import json
import logging

In [5]:
import torch
import numpy as np

In [6]:
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    EvalPrediction
)

In [7]:
from datasets import Dataset
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

<br><br>

#### **2) 변수 및 파라미터 설정**
<br>

In [8]:
output_dir = "/content/output"
model_path = "klue/roberta-base"
tokenizer_path = "klue/roberta-base"
max_seq_len = 512
batch_size = 32
valid_batch_size = 64
accumulate_grad_batches = 1
epochs = 10
learning_rate = 2e-4
weight_decay = 0.01
gpus = 0
seed = 42

<br><br>

#### **3) logging 설정 및 데이터 로드**

In [9]:
logger = logging.getLogger("train")
logger.propagate = False
logger.setLevel(logging.DEBUG)
if not logger.handlers:
    handler = logging.StreamHandler(sys.stdout)
    handler.setFormatter(logging.Formatter("[%(asctime)s] %(message)s"))
    logger.addHandler(handler)

In [10]:
os.makedirs(output_dir, exist_ok=True)
logger.info(f'[+] Save output to "{output_dir}"')

[2023-10-13 12:29:25,574] [+] Save output to "/content/output"


In [12]:
logger.info(f'[+] Load Dataset')
train_ds = Dataset.from_json("/content/nikluge-ea-2023-sample.jsonl")
valid_ds = Dataset.from_json("/content/nikluge-ea-2023-sample.jsonl")

[2023-10-13 12:29:41,893] [+] Load Dataset


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

<br><br>

#### **4) 데이터 전처리**
<br>

In [13]:
logger.info(f'[+] Load Tokenizer"')
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

[2023-10-13 12:29:46,851] [+] Load Tokenizer"


In [14]:
labels = list(train_ds["output"][0].keys())
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

In [15]:
def preprocess_data(examples):
    # take a batch of texts
    text1 = examples["input"]["form"]
    text2 = examples["input"]["target"]["form"]
    # encode them
    encoding = tokenizer(text1, text2, padding="max_length", truncation=True, max_length=max_seq_len)
    # add labels
    encoding["labels"] = [0.0] * len(labels)
    for key, idx in label2id.items():
        if examples["output"][key] == 'True':
            encoding["labels"][idx] = 1.0

    return encoding

In [16]:
encoded_tds = train_ds.map(preprocess_data, remove_columns=train_ds.column_names)
encoded_vds = valid_ds.map(preprocess_data, remove_columns=valid_ds.column_names)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

<br><br>

#### **5) 모델 및 트레이너 설정**
<br>

In [17]:
logger.info(f'[+] Load Model from "{model_path}"')

[2023-10-13 12:29:52,043] [+] Load Model from "klue/roberta-base"


In [18]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    problem_type="multi_label_classification",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<br>

In [19]:
training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=valid_batch_size,
    num_train_epochs=epochs,
    weight_decay=weight_decay,
    load_best_model_at_end=True,
    metric_for_best_model= "f1",
)

In [23]:
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

In [24]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(predictions=preds, labels=p.label_ids)
    return result

In [25]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=encoded_tds,
    eval_dataset=encoded_vds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<br><br>

#### **6) 학습 실행**
<br>

In [26]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.397057,0.666667,0.809524,0.666667
2,No log,0.324675,0.666667,0.809524,0.666667
3,No log,0.265679,0.666667,0.809524,0.666667
4,No log,0.234936,0.666667,0.809524,0.666667
5,No log,0.208426,0.857143,0.976190,0.666667
6,No log,0.184087,1.000000,1.000000,1.000000
7,No log,0.166122,1.000000,1.000000,1.000000
8,No log,0.154278,1.000000,1.000000,1.000000
9,No log,0.146681,1.000000,1.000000,1.000000
10,No log,0.142836,1.000000,1.000000,1.000000


TrainOutput(global_step=10, training_loss=0.2660742998123169, metrics={'train_runtime': 87.293, 'train_samples_per_second': 0.344, 'train_steps_per_second': 0.115, 'total_flos': 7893756887040.0, 'train_loss': 0.2660742998123169, 'epoch': 10.0})

<br>

In [48]:
with open(os.path.join(output_dir, "label2id.json"), "w") as f:
    json.dump(label2id, f)

<br><br>

## **2. inference.py**
<br>

#### **1) 필요 라이브러리 및 패키지 import**
<br>

In [27]:
import json
import logging
import os
from tqdm import tqdm

In [28]:
import numpy as np
import torch

In [29]:
from torch.utils.data import DataLoader
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer

<br>

#### **2) 설정 변수 및 파라미터**
<br>

In [49]:
model_ckpt_path = "/content/output/checkpoint-10"  # ####에는 저장된 체크포인트 번호를 넣어주세요.
output_path = "/content/inference_results.jsonl"
batch_size = 32
max_seq_len = 512
threshold = 0.5
num_beams = 3
device = "cuda" if torch.cuda.is_available() else "cpu"

<br>

#### **3) 데이터 및 모델 로딩**
<br>

In [50]:
logger = logging.getLogger("inference")
logger.propagate = False
logger.setLevel(logging.DEBUG)
if not logger.handlers:
    handler = logging.StreamHandler()
    handler.setFormatter(logging.Formatter("[%(asctime)s] %(message)s"))
    logger.addHandler(handler)

In [51]:
logger.info(f"[+] Use Device: {device}")

[2023-10-13 12:54:53,933] [+] Use Device: cuda


In [52]:
!ls {model_ckpt_path}

added_tokens.json  rng_state.pth	    tokenizer.json
config.json	   scheduler.pt		    trainer_state.json
optimizer.pt	   special_tokens_map.json  training_args.bin
pytorch_model.bin  tokenizer_config.json    vocab.txt


In [46]:
logger.info(f'[+] Load Tokenizer from "{model_ckpt_path}"')
tokenizer = AutoTokenizer.from_pretrained(model_ckpt_path, local_files_only=True)

[2023-10-13 12:46:41,237] [+] Load Tokenizer from "/content/output/checkpoint-1"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [53]:
logger.info(f'[+] Load Dataset')
test_ds = Dataset.from_json("/content/nikluge-ea-2023-sample.jsonl")
with open(os.path.join(model_ckpt_path, "..", "label2id.json")) as f:
    label2id = json.load(f)
labels = list(label2id.keys())
id2label = {v: k for k, v in label2id.items()}

[2023-10-13 12:55:01,247] [+] Load Dataset


In [54]:
def preprocess_data(examples):
    text1 = examples["input"]["form"]
    text2 = examples["input"]["target"]["form"]
    encoding = tokenizer(text1, text2, padding="max_length", truncation=True, max_length=max_seq_len)
    return encoding

In [55]:
encoded_tds = test_ds.map(preprocess_data, remove_columns=test_ds.column_names).with_format("torch")
data_loader = DataLoader(encoded_tds, batch_size=batch_size)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [56]:
logger.info(f'[+] Load Model from "{model_ckpt_path}"')

[2023-10-13 12:55:34,889] [+] Load Model from "/content/output/checkpoint-10"


In [57]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_ckpt_path,
    problem_type="multi_label_classification",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
)

In [58]:
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

<br><br>

#### **4) 추론 및 결과 저장**
<br>

In [59]:
logger.info("[+] Start Inference")

[2023-10-13 12:57:22,845] [+] Start Inference


In [60]:
sigmoid = torch.nn.Sigmoid()
outputs = []

In [61]:
for batch in tqdm(data_loader):
    oup = model(
        batch["input_ids"].to(device),
        token_type_ids=batch["token_type_ids"].to(device),
        attention_mask=batch["attention_mask"].to(device)
    )

    probs = sigmoid(oup.logits).cpu().detach().numpy()
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1

    outputs.extend(y_pred.tolist())

100%|██████████| 1/1 [00:00<00:00,  6.30it/s]


In [62]:
j_list = test_ds
for idx, oup in enumerate(outputs):
    j_list[idx]["output"] = {}
    for jdx, v in enumerate(oup):
        j_list[idx]["output"][id2label[jdx]] = "True" if v else "False"

In [63]:
with open(output_path, "w", encoding='utf-8') as f:
    for json_data in j_list:
        f.write(json.dumps(json_data, ensure_ascii=False)+'\n')